In [1]:
from block_methods import *
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker, cm
import matplotlib.colors as colors
from scipy import integrate

%load_ext autoreload
%autoreload 2

In [2]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis([0, 0.6, 0.8]))

In [3]:
n = 1000 # matrix dimension
b = 4 # block size
step = 5
thresh = 1e-8

# set up matrix
Λ = np.linspace(1e-2,1,n)
H = np.diag(Λ)

λmin = np.min(Λ)
λmax = np.max(Λ)

f = lambda x: np.sqrt(x)

K = 100 # max iter
w = 0

angles = (1-np.array([0.01, 0.1, 0.5, 0.9]))*np.pi # angle for D contour
r = [1, 1.01, 1.05, 1.1] # radius for D contour
lim = 10
np.random.seed(0)

hnorm = get_hnorm(Λ, lambda Λ: Λ-w)

In [4]:
error_FAr = []
error_wLSr = []
error_absr = []
error_fullr = []

V = np.random.randn(n,b)

# run Lanczos
Q,Qkp1,A,B,B_0 = block_lanczos(H, V, K, K+1)

# generate tridiagonal matrix
T = get_block_tridiag(A,B)

orthTest(Q, b, K, thresh)
threeTermTest(H, Q, T, Qkp1, B, b, K, thresh)

import time
s = time.time()

fAV = np.diag(f(Λ))@V
for i in range(len(r)):
    for l in range(len(angles)):
        error_FA = np.full(K,np.nan)
        error_wLS = np.full(K,np.nan)
        error_abs = np.full(K,np.nan)
        error_full = np.full(K,np.nan)

        lsT = np.full(K//step,0, dtype = float)
        FAT = np.full(K//step,0, dtype = float)
        absT = np.full(K//step,0, dtype = float)
        fullT = np.full(K//step,0, dtype = float)

        for k in range(1,K,step): 

            T = get_block_tridiag(A[:k],B[:k])
            Eval,Evec = np.linalg.eigh(T)

            error_wLS[k] = hnorm(np.diag(1/(Λ-w))@V - get_lan_wLS(Eval, Evec, b, B_0, w, Q, k))

            lanf = Q[:,:b*k]@(Evec@np.diag(f(Eval))@Evec.T@Ei(b*k,b,1)@B_0)
            error_FA[k] = hnorm(fAV - lanf)
            
            pts = np.logspace(-15, -1, lim)
            
            error_abs[k] = get_trig_ineq_bound(pts, angles[l], r[i], Eval, Evec, b, B_0, λmin, f, λmin/100, Λ, V, Q, k, hnorm)
            error_full[k] = get_a_posteriori_bound(pts, angles[l], r[i], Eval, Evec, b, B_0, λmin, f, λmin/100, w, λmax)

        error_FAr.append(error_FA)
        error_wLSr.append(error_wLS)
        error_absr.append(error_abs)
        error_fullr.append(error_full)
        
print(time.time()-s)

/mnt/c/Users/qcx/Conda/block_lanczos_CIF/block_methods/block_methods.py:391: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result += sp.integrate.quad(a_posteriori_bound_integrand, 0, 1, args=(Γl, angle, r, Eval, Evec, b, B_0, λmin, f, c, w, λmax), points = pts)[0]
/mnt/c/Users/qcx/Conda/block_lanczos_CIF/block_methods/block_methods.py:383: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result = sp.integrate.quad(trig_ineq_bound_integrand, 0, angle, args=(Γ, angle, r, Eval, Evec, b, B_0, λmin, f, c, Λ, V, Q, k, hnorm))[0]
/mnt/c/Users/qcx/Conda/blo

219.14344382286072


In [5]:
np.save("data/figure_3.4/error_FAr", error_FAr)
np.save("data/figure_3.4/error_wLSr", error_wLSr)
np.save("data/figure_3.4/error_absr", error_absr)
np.save("data/figure_3.4/error_fullr", error_fullr)

# Uncomment the cell below to generate figures directly from data. 

In [6]:
# error_FAr = np.load("data/figure_3.4/error_FAr.npy")
# error_wLSr = np.load("data/figure_3.4/error_wLSr.npy")
# error_absr = np.load("data/figure_3.4/error_absr.npy")
# error_fullr = np.load("data/figure_3.4/error_fullr.npy")

In [7]:
fig, axes = plt.subplots(len(r), len(angles), figsize=(9,9), sharex=True, sharey=True)

plt.subplots_adjust(wspace=.1, hspace=.1)

for i in np.arange(len(r)):
    for j in np.arange(len(angles)):
        axes[i, j].plot(error_FAr[i*len(angles)+j], ls='None', ms=4, marker = 's', label = 'Lanczos-FA error')
        axes[i, j].plot(error_absr[i*len(angles)+j], ls='None',ms=4, marker = 'd', label = 'triangle inequality')
        axes[i, j].plot(error_fullr[i*len(angles)+j]*error_wLSr[i*len(angles)+j], ls='None',ms=4, marker = 'o', label = 'computable bound')
        handles, labels = axes[i, j].get_legend_handles_labels()
        plt.yscale('log')
        
        axes[i, j].text(.95,.95,f'$\\Theta={angles[j]/np.pi:1.2f}\\pi, R={r[i]:1.2f}$',ha='right', va='top', transform=axes[i, j].transAxes,bbox=dict(facecolor='white',edgecolor='none',pad=2))

plt.setp(axes[-1, :], xlabel='Iteration')
plt.setp(axes[:, 0], ylabel='Error')
 
plt.figlegend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 0),ncol=3)

plt.savefig("imgs/figure_3.4.pdf")

RuntimeError: Failed to process string with tex because latex could not be found

Error in callback <function _draw_all_if_interactive at 0x7ff2bdeaadd0> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 900x900 with 16 Axes>